In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch

In [21]:
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [37]:
MNISTNet

__main__.MNISTNet

In [13]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

In [14]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )

In [15]:
def train_model():
    device = "cpu"
    epochs = 1
    train_kwargs = {"batch_size": 256}
    test_kwargs = {"batch_size": 1000}
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
    dataset1 = datasets.MNIST("../data", train=True, download=True, transform=transform)
    dataset2 = datasets.MNIST("../data", train=False, transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)
    model = MNISTNet().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=1.0)

    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()
    return model

In [16]:
my_model = train_model()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295776
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.631971
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.526925
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.409722
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.225346
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.312465
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.240750
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.189270
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.211113
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.132681
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.164609
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.174042
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.186887
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.181484
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.135922
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.108739
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.066518
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.116501
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.203046
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.076458
Train Epoc

In [38]:
my_model

MNISTNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [17]:
from slashml import ModelDeployment

In [18]:
# Replace `API_KEY` with your SlasML API token.
client = ModelDeployment(api_key="68b50f46b97f8c0f98fe419df240a22af3da392f")

In [19]:
response = client.deploy(model_name='pytorch-model', model=my_model, requirements=['torch', 'transformers'])

> /Users/faizank/workspace/experiments/live_projects/slashml/sdk_tests/venv-3.9.1/lib/python3.9/site-packages/truss/model_frameworks/pytorch.py(61)_torch_package()
     59         import pdb
     60         pdb.set_trace()
---> 61         exp.save_pickle(TORCH_MODEL_PACKAGE_NAME, TORCH_MODEL_PICKLE_FILENAME, model)
     62 
     63 



ipdb>  c


> /Users/faizank/workspace/experiments/live_projects/slashml/sdk_tests/venv-3.9.1/lib/python3.9/site-packages/torch/package/package_exporter.py(1090)close()
   1088         import pdb
   1089         pdb.set_trace()
-> 1090         self._execute_dependency_graph()
   1091         self._write_python_version()
   1092 



ipdb>  c


In [24]:
client.status(model_version_id=response.id)

{'id': '509107b5-6454-4af6-8be2-3be2a8d379a6',
 'created': '2023-09-26T22:58:16.103933Z',
 'status': 'READY',
 'name': 'pytorch-model'}

In [26]:
status = client.status(model_version_id=response.id)

while status.status != 'READY':
    print(f'status: {status.status}')
    print('trying again in 5 seconds')
    time.sleep(5)
    status = client.status(model_version_id=response.id)

    if status.status == 'FAILED':
        raise Exception('Model deployment failed')

print('model ready')

model ready


In [28]:
# prepare data for inference
transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
inputs = datasets.MNIST("../data", train=False, transform=transform)
dataset = torch.utils.data.DataLoader(inputs, batch_size=1)

In [29]:
model_input = list(dataset)[0]

In [32]:
inputs = torch.tensor(
                model_input[0], dtype=list(my_model.parameters())[0].dtype, device='cpu'
            )

/var/folders/qg/6sb9klt17nbc4_jz2gx49s240000gn/T/ipykernel_64542/2345019260.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(


In [33]:
prediction = client.predict(model_version_id=response.id, model_input=inputs.tolist())


In [34]:
scores = prediction['model_response']['predictions']


In [35]:
scores

[[-13.912128448486328,
  -14.697678565979004,
  -9.851377487182617,
  -11.565227508544922,
  -16.64999771118164,
  -17.572322845458984,
  -22.370956420898438,
  -6.937739817658439e-05,
  -14.558772087097168,
  -12.155047416687012]]

In [15]:
'__file__' in globals()

False

In [16]:
if '__file__' not in globals():
    __file__ = 'temporary_notebook_file.py'

In [12]:
if __name__=='__main__':
    print('something')

something


In [17]:
from torch import package
import os
os.makedirs('test', exist_ok=True)
target_directory = os.getcwd()+"/test/"
TORCH_PACKAGE_FILE = "model_package.pt"
TORCH_MODEL_PICKLE_FILENAME = "model.pkl"
TORCH_MODEL_PACKAGE_NAME = "torch_model"
path = target_directory+f"/{TORCH_PACKAGE_FILE}"

extern_modules=["__main__"]
with package.PackageExporter(path, debug=True) as exp:
    exp.intern(f"{my_model.__class__.__module__}.**")
    # for extern_module in extern_modules:
    #     exp.extern(f"{extern_module}.**")
    exp.save_pickle(TORCH_MODEL_PACKAGE_NAME, TORCH_MODEL_PICKLE_FILENAME, my_model)

> /Users/faizank/workspace/experiments/live_projects/slashml/sdk_tests/venv-3.9.1/lib/python3.9/site-packages/torch/package/package_exporter.py(1090)close()
   1088         import pdb
   1089         pdb.set_trace()
-> 1090         self._execute_dependency_graph()
   1091         self._write_python_version()
   1092 



ipdb>  c


In [11]:
model_py_string = """
import torch
import torch.nn as nn
import torch.nn.functional as F

class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
"""

with open('model.py', 'w') as model_py_file:
    model_py_file.write(model_py_string)

In [12]:
from model import MNISTNet


In [20]:
_

''